In [44]:
# Load pandas library
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [45]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
sns.set(color_codes= True)

### Exploratory Data Analysis 

In [ ]:
# Import first csv: year 2021
df1_import = pd.read_csv('atlantic_sunrise2021.csv',  
                           sep=',',
                           engine='python',
                           na_values = ['NA', 'N/A','NaN'],
                           header=None,
                           skiprows = [0],
                           index_col= False,
                           names = ['customer_id', 'First_Name', 'last_name', 'address', 'City', 'county', 'state',
                           'ZIP', 'customer_phone', 'Fax_Number', 'Customer_Email', 'Rental_ID', 'Weekly_Rate', 'Parking',
                           'Pool_Access', 'Repeat_Customer', 'Bedrooms', 'MGMT_Discount', 'Date','total_sale',])  # Read first CSV file)
                           
df1_import.head().append(df1_import.tail())

In [ ]:
# Scan for duplicates
df1_import.duplicated()

In [ ]:
# Passing the "last" argument: mark the first occurrence of the value as non-duplicate
df1_import.duplicated(keep="last")

In [ ]:
# Data len
len(df1_import), len(df1_import.drop_duplicates())

In [ ]:
# Remove duplicated data
df1_import = df1_import.drop_duplicates()
df1_import

In [ ]:
# Import second csv: year 2022
df2_import = pd.read_csv('atlantic_sunrise2022.csv',
                           sep=',',
                           engine='python',
                           na_values = ['NA', 'N/A', 'NaN'],
                           header=None,
                           skiprows = [0],
                           index_col= False,
                           names = ['customer_id', 'First_Name', 'last_name', 'address', 'City', 'county', 'state',
                           'ZIP', 'customer_phone', 'Fax_Number', 'currency', 'Customer_Email', 'Rental_ID', 'Weekly_Rate', 'Parking',
                           'Pool_Access', 'Repeat_Customer','Bedrooms', 'MGMT_Discount', 'Date', 'total_sale'])    # Read second CSV file

df2_import.head().append(df2_import.tail())

In [ ]:
# Skim through duplicates
df2_import.duplicated() 

In [53]:
# Passing the "last" argument: for second csv
df2_import.duplicated(keep="last")

In [54]:
#  Data len
len(df2_import), len(df2_import.drop_duplicates())

In [ ]:
# Remove duplicated data
df2_import = df2_import.drop_duplicates()
df2_import

In [ ]:
# Changing columns name to lowercase text (csv 1)
df1_import.columns = map(str.lower,df1_import.columns)

df1_import.columns = map(lambda x: x.replace("-", "_").replace(" ", "_"), df1_import.columns)

df1_import.columns

In [ ]:
# Changing columns name to lowercase text (csv 2)
df2_import.columns = map(str.lower,df2_import.columns)

df2_import.columns = map(lambda x: x.replace("-", "_").replace(" ", "_"), df2_import.columns)

df2_import.columns

In [ ]:
# Get dtype info for each column (csv 1)
df1_import.info()

In [ ]:
# Convert data type for the selected columns in a list (str) (csv 1)
convert_data = pd.DataFrame(df1_import)
cols = ['customer_id', 'first_name', 'last_name', 'address', 'city', 'county','state','zip',
        'customer_phone', 'fax_number', 'customer_email', 'rental_id','bedrooms']
convert_data[cols] = convert_data[cols].astype('str')

In [ ]:
# Using a for loop to convert some columns (bool) (csv 1)
for col in ['parking','pool_access','repeat_customer', 'mgmt_discount']:
    df1_import[col] = df1_import[col].astype('bool_')

In [ ]:
# Convert weekly rate (int) (csv1)
df1_import["weekly_rate"] = pd.to_numeric(df1_import.weekly_rate, errors = 'coerce', downcast ='integer' )

In [ ]:
# Using apply() & astype() together to convert date column (datetime64[ns]) (csv 1)
df1_import[['date']].apply(lambda x: x.astype('datetime64[ns]'))

In [ ]:
# Get dtype info for each column (csv 2)
df2_import.info()

In [ ]:
# Convert data type for the selected columns in a list (str) (csv 2)
convert_data = pd.DataFrame(df2_import)
cols = ['customer_id', 'first_name', 'last_name', 'address', 'city', 'county','state','zip',
        'customer_phone', 'fax_number', 'currency', 'customer_email', 'rental_id','bedrooms']
convert_data[cols] = convert_data[cols].astype('str')

In [65]:
# Using a for loop to convert some columns (bool) (csv 2)
for col in ['parking','pool_access','repeat_customer', 'mgmt_discount']:
    df2_import[col] = df2_import[col].astype('bool_')

In [66]:
# Convert weekly rate (int) (csv 2) 
df2_import= df2_import.astype({"weekly_rate": int},errors='ignore')

In [ ]:
# Using apply() & astype() together to convert date column (datetime64[ns]) (csv 2)
df2_import[['date']].apply(lambda x: x.astype('datetime64[ns]'))

In [68]:
# Added new column to dataframe (csv 1)
df1_import['year'] = '2021'
df1_import

In [ ]:
# Added new column to dataframe (csv 2)
df2_import['year'] = '2022'
df2_import

In [ ]:
# Combined two datasets by .merge()
resort_df = df1_import.merge(df2_import,
                              how='outer', 
                              right_index= False,
                              left_index= False)
resort_df

In [71]:
# Count missing values by columns
resort_df.isnull().sum()

In [ ]:
# Fill 'currency' column with 'USD'
resort_df.fillna('USD', inplace=True)

In [ ]:
# Replace 'None' with correct input
resort_df["currency"] = resort_df['currency'].replace('None',  value='USD')
resort_df

In [ ]:
# Identify unique value in 'rental_id' columns
resort_df.rental_id.unique()

In [ ]:
# Remove column from a list of columns (iteratively) by condition.
for col in resort_df.columns:
    if 'rental_id' in col:
        del resort_df[col]

resort_df

In [ ]:
# Insert new column 'rent_discount' (15% off from 'weekly_rate' discount if renters rent for more than a week)
rent_discount = []
for value in resort_df['mgmt_discount']:
    if value == True:
        rent_discount.append(0.85)
    elif value == False:
        rent_discount.append(0)
    else:
        rent_discount.append('None')

resort_df["rent_discount"] = rent_discount
resort_df

In [ ]:
# Machine learning
type(True)
type(False)
issubclass(bool, int)
True == 1
False == 0

In [ ]:
# Rename multiple columns
resort_df.rename(columns = {'fax_number':'invoice_id',
                            'customer_email':'personal_email',
                            'date': 'check_in'}, inplace= True)
resort_df.columns

In [ ]:
# Verify if any value is null
resort_df.isnull().values.any() 

In [ ]:
# Using describe 'all' method to get statistical characteristics of each numerical/non-numericals columns. 
resort_df.describe(include='all')

In [ ]:
# Manual method for non-numeric columns 
resort_df.describe(include=['object', 'object'])